In [ ]:
!pip install pinecone-client qianfan tiktoken langchain pymupdf

### pinecone初始化

初始化pinecone，并创建pinecone索引（index）

In [29]:
import getpass
PINECONE_API_KEY = getpass.getpass("input your pinecone api key:")
PINECONE_ENV = input("input your env name")

In [17]:
import pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

index_name = "qianfan-vdb"

if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
        name=index_name,
        metric='euclidean',
        dimension=384,  # 1536 dim of text-embedding-ada-002
    )

使用千帆SDK前需要进行初始化鉴权

In [6]:
import os 

os.environ["QIANFAN_AK"] = getpass.getpass("input your qianfan app ak:")
os.environ["QIANFAN_SK"] = getpass.getpass("input your qianfan app sk:")

# s.environ["QIANFAN_ACCESS_KEY"] iam


嵌入准备，实际上将text转换成高维向量的表示，Embeddings对象基于qianfan 实现

In [30]:
from langchain.embeddings import QianfanEmbeddingsEndpoint
embeddings = QianfanEmbeddingsEndpoint(model="Embedding-V1")


初始化vectorstore，使用PyMuPDFLoader加载PDF文件

In [31]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("./example_data/ai-paper.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=384, chunk_overlap=0, separators=["\n\n", "\n", " ", "", "。", "，"])
docs_spilts = text_splitter.split_documents(documents)
print(f"{len(docs_spilts)} documents block loaded")

pinecone_vdb = Pinecone.from_documents(docs_spilts, embeddings, index_name=index_name)

33 documents block loaded


In [32]:
#新建Retriever
retriever = pinecone_vdb.as_retriever(search_type="similarity_score_threshold", search_kwargs={'score_threshold': 0.5, "k":3 })
matched_docs = retriever.get_relevant_documents("DeepMind")



/Users/zhonghanjun/anaconda3/lib/python3.11/site-packages/langchain/schema/vectorstore.py:313: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='放。DeepMind 指出，在对训练数据集进行扩展\n时，需要重点关注数据集的质量管理，尤其是其中\n的伦理和隐私等问题。\n通信企业管理\u2003 C-Enterprise\u2003Management\u2003 2023.6\u2003 9', metadata={'author': 'CNKI', 'creationDate': "D:20230718144804-08'00'", 'creator': 'ReaderEx_DIS 2.3.0 Build 4031', 'file_path': './example_data/ai-paper.pdf', 'format': 'PDF 1.6', 'keywords': '', 'modDate': '', 'page': 3.0, 'producer': 'TTKN', 'source': './example_data/ai-paper.pdf', 'subject': '', 'title': '', 'total_pages': 7.0, 'trapped': ''}), 1.00852227), (Document(page_content='放。DeepMind 指出，在对训练数据集进行扩展\n时，需要重点关注数据集的质量管理，尤其是其中\n的伦理和隐私等问题。\n通信企业管理\u2003 C-Enterprise\u2003Management\u2003 2023.6\u2003 9', metadata={'author': 'CNKI', 'creationDate': "D:20230718144804-08'00'", 'creator': 'ReaderEx_DIS 2.3.0 Build 4031', 'file_path': './example_data/ai-paper.pdf', 'format': 'PDF 1.6'

In [33]:
# 新建LLM类对象
from langchain.chat_models import QianfanChatEndpoint

llm = QianfanChatEndpoint(model="ERNIE-Bot-4")

In [27]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

CUSTOM_PROMPT = """
现在你是一个阅读理解机器人，你会阅读并深度理解我给你的文本内容并据此回答我所提出的问题。注意，我给出的问题是：{question} 你需要阅读理解的文本是：{context}
"""


qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever, chain_type_kwargs={"prompt": PromptTemplate.from_template(CUSTOM_PROMPT)}, return_source_documents=True)

In [34]:

query = "ERNIE1.0是什么时候发布的？" 
qa_chain({"query": query})

/Users/zhonghanjun/anaconda3/lib/python3.11/site-packages/langchain/schema/vectorstore.py:313: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='勇在阿里云峰会上正式发布了大语言模型工具通义\n千问，并宣布此语言模型会接入阿里旗下的所有业\n务中。\n百度 ERNIE\n百 度 于 2019 年 3 月 发 布 预 训 练 模 型\nERNIE1.0，2019 年 7 月 发 布 ERNIE2.0，2021\n年 5 月开源四大预训练模型，包括多粒度语言知\n识模型 ERNIE-Gram、超长文本双向建模预训练\n模型 ERNIE-Doc、融合场景图知识的跨模态预训\n练模型 ERNIE-ViL 和语言与视觉一体的预训练\n模型 ERNIE-UNIMO，2021 年 12 月发布多语言\n预训练模型 ERNIE-M。百度持续投入大模型的\n技术创新与产业应用，布局了 NLP、CV、跨模态\n等大模型，率先提出行业大模型，构建大模型工具\n与平台，探索产品与社区，在企业端和用户端均有', metadata={'author': 'CNKI', 'creationDate': "D:20230718144804-08'00'", 'creator': 'ReaderEx_DIS 2.3.0 Build 4031', 'file_path': './example_data/ai-paper.pdf', 'format': 'PDF 1.6', 'keywords': '', 'modDate': '', 'page': 5.0, 'producer': 'TTKN', 'source': './example_data/ai-paper.pdf', 'subject': '', 'title': '', 'total_pages': 7.0, 'trapped': ''}), 1.07225263), (Document(page_content='勇在阿里云峰会上正式发布了大语言模型工具通义\n千问，并宣布此语言模型会接入阿里旗下的所有业\n务中。\n百度 E

{'query': 'ERNIE1.0是什么时候发布的？',
 'result': '根据提供的文本内容，百度于2019年3月发布了预训练模型ERNIE1.0。因此，ERNIE1.0的发布时间是2019年3月。',
 'source_documents': [Document(page_content='勇在阿里云峰会上正式发布了大语言模型工具通义\n千问，并宣布此语言模型会接入阿里旗下的所有业\n务中。\n百度 ERNIE\n百 度 于 2019 年 3 月 发 布 预 训 练 模 型\nERNIE1.0，2019 年 7 月 发 布 ERNIE2.0，2021\n年 5 月开源四大预训练模型，包括多粒度语言知\n识模型 ERNIE-Gram、超长文本双向建模预训练\n模型 ERNIE-Doc、融合场景图知识的跨模态预训\n练模型 ERNIE-ViL 和语言与视觉一体的预训练\n模型 ERNIE-UNIMO，2021 年 12 月发布多语言\n预训练模型 ERNIE-M。百度持续投入大模型的\n技术创新与产业应用，布局了 NLP、CV、跨模态\n等大模型，率先提出行业大模型，构建大模型工具\n与平台，探索产品与社区，在企业端和用户端均有', metadata={'author': 'CNKI', 'creationDate': "D:20230718144804-08'00'", 'creator': 'ReaderEx_DIS 2.3.0 Build 4031', 'file_path': './example_data/ai-paper.pdf', 'format': 'PDF 1.6', 'keywords': '', 'modDate': '', 'page': 5.0, 'producer': 'TTKN', 'source': './example_data/ai-paper.pdf', 'subject': '', 'title': '', 'total_pages': 7.0, 'trapped': ''}),
  Document(page_content='勇在阿里云峰会上正式发布了大语言模型工具通义\n千问，并宣布此语言模型会接入阿里旗下的所有业\n务中。\n百度 ERNIE\n百 度 于 2019 年 3 月 发 布 预 训 练 模 型\n